# atomman.nlist(system, cutoff, cmult=1)

- - -

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2016-09-02

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

The nlist function builds and returns a neighbor list array for all atoms in a system. The function uses binning to speed up the neighbor search, and vectorized numpy routines to reduce calculation time.

Arguments:

- __system__ -- the [System](atomman.System.ipynb) to construct the neighbor list for.

- __cutoff__ -- the cutoff distance to use in identifying the neighbors.

Keyword Argument:

- __cmult__ -- integer parameter that affects the size of bins. Default value is most likely the fastest.

The returned array is of size natomsXnvals where nvals is at least one larger than the maximum number of neighbors (coordination) for all of the atoms. Each row of the array is for a different atom, where the first term indicates the atom's coordination, and the rest of the terms the atom ids of the neighbors. Any extra values in the array will have values of zero.

### Routine Description

The routine first identifies an orthogonal super box that fully encompases the system with a cushion of the cutoff distance in all directions. Next, the super box is divided into square bins with dimensions equal to cutoff/cmult. All of the atoms in system are assigned to bins. Ghost atoms corresponding to images of the real atoms across periodic boundaries and are contained in the super box are generated and also assigned to bins. Every single atom in a bin with real atoms is compared with atoms in the same bin and nearby bins, and any pairs of atoms closer than the cutoff distance are appended to each other's neighbor list. 

For the binning algorithm, cmult affects the volume of space that needs to be explored for identifying neighbors. With a cmult value of 1, the bin size equals the cutoff distance and the search for neighbors checks the current bin and bins +-1 around it. Increasing cmult to 2 decreases the volume (number of atoms) of the search, but increases the number of bins (to +-2) to search. A cmult of 1 seems to be the fastest for a typical nearest neighbors calculation with the code used here.

In terms of calculation speed, Python can be notably slower than other languages like c++ for loop-heavy algorithms. The use of numpy routines has greatly reduced the calculation time compared to pure Python, but improvements are still possible.

The underlying code can be found in [atomman/core/nlist.py](../../atomman/core/nlist.py).

## Demonstration

In [1]:
import atomman as am
import numpy as np

Generate a demonstration fcc 5x5x5 cell

In [2]:
alat = 3.64
atype = 1
pos = np.array([[0.0, 0.0, 0.0], [0.5, 0.5, 0.0], [0.5, 0.0, 0.5], [0.0, 0.5, 0.5]])

atoms = am.Atoms(natoms=len(pos), prop={'atype':atype, 'pos':pos})
box = am.Box(a=alat, b=alat, c=alat)
system = am.System(atoms=atoms, box=box, scale=True)

system.supersize(5,5,5)
print system.natoms

500


Compute the nearest neighbor list

In [3]:
cutoff = 0.8 * alat
nlist = am.nlist(system, cutoff)

### Getting values from nlist

Coordination values are stored as the first term in every row of nlist

In [4]:
#Compute average coordination
ave_coord = np.mean(nlist[:,0])
print ave_coord

12.0


Atom ids of the neighbor atoms are stored after the coordination number. 

In [6]:
#List all neighbor atom ids for atom 100
print nlist[100, 1 : nlist[100, 0] + 1]

[  2   3  18  83 101 102 103 117 118 181 183 197]


__WARNING!__ The array column is likely longer than absolutely necessary, with the extra values filled up with zeros. Be sure to not access values in a column with an index greater than the coordination number. 

In [7]:
print nlist[100, 1:]

[  2   3  18  83 101 102 103 117 118 181 183 197   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]


- - -
__Docs Navigation:__

Tutorial:

1. [Basics](../tutorial/1 Basics.ipynb)

2. [LAMMPS Functionality](../tutorial/2 LAMMPS Functionality.ipynb)

3. [Defect Generation and Evaluation](../tutorial/3 Defect Generation and Evaluation.ipynb)


Reference:

- [atomman](../reference/atomman.ipynb)

- [atomman.convert](../reference/atomman.convert.ipynb)

- [atomman.defect](../reference/atomman.defect.ipynb)

- [atomman.lammps](../reference/atomman.lammps.ipynb)

- [atomman.tools](../reference/atomman.tools.ipynb)

- [atomman.unitconvert](../reference/atomman.unitconvert.ipynb)